# Create Embedding
In this notebook, we'll connect to a Neo4j instance.  We'll load data based on a schema and compute graph embeddings.  The notebook exports that data to pandas and then writes them to Cloud Storage as CSV files.

## Using the Neo4j API
Let's connect to our Neo4j deployment.  First off, install the Neo4j Graph Data Science package.

In [ ]:
!pip install graphdatascience --quiet
!pip install --quiet google-cloud-storage
!pip install --quiet google.cloud.aiplatform

Now, you're going to need the connection string and credentials from the deployment you created above.

In [ ]:
# Edit these variables!
DB_URL = '' #'neo4j+s://URL.databases.neo4j.io'
DB_PASS = ''

# You can leave this default
DB_USER = 'neo4j'

Lets create GDS connection object using the variables defined above

In [ ]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(DB_URL, auth=(DB_USER, DB_PASS))

# Explore & Load Data

The dataset we are going to use is from a public available [Kaggle dataset](https://www.kaggle.com/datasets/rohitrox/healthcare-provider-fraud-detection-analysis).  These are healthcare expense claims with anonymised beneficiaries, claims and providers.  We've filtered the data and cleaned up the datasets. The cleaned data can be downloaded [here](https://storage.googleapis.com/neo4j-datasets/insurance-claim/data.csv)

We will predict the potentially fraudulent providers based on the claims filed by them. We will use a GDS embedding algorithm to chart out Fraudulent patterns in the provider's claims to understand the future behaviour of providers.

The dataset has

- **Inpatient Data**: 
Contains claims filed for those patients who are admitted in the hospitals. It also provides additional details like their admission and discharge dates and admit and diagnosis code.
- **Outpatient Data**
- **Beneficiary Details Data**: 
Contains beneficiary KYC details like health conditions,regioregion they belong to etc. 


Before loading data into any Database, we usually have to come up with a schema and implement it on the Database. Graph Data Modelling is an important step with Neo4j and you define it based on the questions you would like to ask on the graph.

If you are interested, you can explore the data using Pandas

In [ ]:
import pandas as pd
import numpy as np

#Let
raw_df = pd.read_csv('https://storage.googleapis.com/neo4j-datasets/insurance-claim/data.csv', 
                     index_col=False, dtype='unicode', parse_dates=['claim_start_dt', 'claim_end_dt', 'admission_dt', 'discharge_dt'])
raw_df.deductible_amt_paid = raw_df.deductible_amt_paid.astype(float)
raw_df

Let's go with the following schema as our questions are more focussed around the relationships between claims, providers, physicians and the diagnoses 

![image info](https://storage.googleapis.com/neo4j-datasets/insurance-claim/img/schema.png)

### Create Constraints
In order to ensure uniqueness of nodes, lets create some constraints. This will ensure that no duplicate nodes are created and speed up the CSV loading process, especially if we want to use `MERGE` statements - as MERGE statement creates new nodes only if they don't exist. 

In [ ]:
import re

node_labels = ['Beneficiary', 'County', 'State', 'Claim', 'DiagnosisGroup', 'AdmitDiagnosis', 'Physician', 'Provider', 'Diagnosis', 'Procedure']

def to_snake_case(s: str) -> str:
    return re.sub(r'(?<!^)(?=[A-Z])', '_', s).lower()

for node_label in node_labels:
    gds.run_cypher(f'CREATE CONSTRAINT {to_snake_case(node_label) + "_node_key"} IF NOT EXISTS FOR (n:{node_label}) REQUIRE n.uid IS NODE KEY;')

# Unlike above nodes, Condition  will use a "name" property as a unique key.
gds.run_cypher(f'CREATE CONSTRAINT condition_node_key IF NOT EXISTS FOR (n:Condition) REQUIRE n.name IS NODE KEY;')

### Load Data

Now, we're going to take data from the Google Cloud Storage bucket and import it into Neo4j.  There are a few different ways to do this.  We'll do with a naive LOAD CSV statements via the GDS Python API.  

The Neo4j [Data Importer](https://data-importer.neo4j.io/) is another option.  It's a great graphical way to import data.  However, the LOAD CSV option we're using makes it really easy to pull directly from Cloud Storage, so is probably a better choice for what we need.

Lets start creating the `Beneficiary`, `Claim`, `Provider`, `County` and `State` nodes first.

In [ ]:
%%time
def chunks(xs, n=30_000):
    n = max(1, n)
    return [xs[i:i + n] for i in range(0, len(xs), n)]

records = raw_df.to_dict('records')
print('======  loading Beneficiary, Provider, Country, State, and Claim nodes  ======')

cumulative_count = 0
for recs in chunks(records):
    gds.run_cypher(
      """
        UNWIND $records AS row

        MERGE (bene:Beneficiary {uid: row.bene_id})
        ON CREATE SET
            bene.dob = row.dob,
            bene.gender = row.gender,
            bene.race = row.race,
            bene.ipAnnualReimbursementAmt = row.ip_annual_reimbursement_amt,
            bene.opAnnualReimbursementAmt = row.op_annual_reimbursement_amt,
            bene.ipAnnualDeductibleAmt = row.ip_annual_deductible_amt,
            bene.opAnnualDeductibleAmt = row.op_annual_deductible_amt,
            bene.partACovMonths = row.num_of_months_part_a_cov,
            bene.partBCovMonths = row.num_of_months_part_b_Cov,
            bene.dod = row.dod

        MERGE (provider:Provider {uid: row.provider})

        MERGE (county:County {uid: row.county})

        MERGE (state:State {uid: row.state})

        MERGE (claim:Claim {uid: row.claim_id})
            SET claim.startDate = row.claim_start_dt,
                claim.endDate = row.claim_end_dt,
                claim.reimbursedAmt = row.claim_amt_reimbursed,
                claim.isFraud = row.is_fraud,
                claim.dischargeDate = row.discharge_dt,
                claim.admitDate = row.admission_dt,
                claim.deductibleAmtPaid = row.deductible_amt_paid
      """
    , params={'records': recs})
    cumulative_count += len(recs)
    print(f'Loaded {cumulative_count:,} of {len(records):,} records')
print(f'Loading Complete')

As per the schema we agreed upon earlier, lets start to connect the ndoes we just created

In [ ]:
records = raw_df[['bene_id', 'claim_id', 'provider', 'county', 'state']].to_dict('records')

result = gds.run_cypher(
  """
    UNWIND $records AS row
    MATCH (bene:Beneficiary {uid: row.bene_id})
    MATCH (claim:Claim {uid: row.claim_id})
    MATCH (provider:Provider {uid: row.provider})
    MATCH (county:County {uid: row.county})
    MATCH (state:State {uid: row.state})

    MERGE (county)<-[:LOCATED_AT]-(bene)
    MERGE (bene)-[:FILED_CLAIM]->(claim)-[:PROVIDED_BY]->(provider)
    MERGE (state)<-[:PART_OF]-(county)
  """
    , params={'records': records})

Time to create the `Physician` nodes and relate them to Claims and Providers

In [ ]:
# create a reshaped physician dataframe to make loading more efficient
physician_role_map = {'operating_physician':'OPERATED_BY', 'attending_physician':'ATTENDED_BY', 'other_physician':'ALSO_ATTENDED_BY'}
physician_dfs = []
for col, role in physician_role_map.items():
    temp_df = raw_df[['claim_id', 'provider', col]].rename(columns={col: 'physician_id'}).dropna()
    temp_df['physician_role'] = role
    physician_dfs.append(temp_df)
physician_df = pd.concat(physician_dfs)
physician_df

In [ ]:
# Load Physician nodes
gds.run_cypher(
    """
      UNWIND $physicianIds AS physicianId
      MERGE (physician:Physician {uid: physicianId})
    """
    , params={'physicianIds': physician_df.physician_id.drop_duplicates().tolist()})

In [ ]:
# Load Physician worked for provider relationship
gds.run_cypher(
    """
      UNWIND $records AS row
      MATCH (provider:Provider {uid: row.provider})
      MATCH (physician:Physician {uid: row.physician_id})
      MERGE (provider)<-[:WORKS_FOR]-(physician)
    """
    , params={'records': physician_df[['provider','physician_id']].drop_duplicates().to_dict('records')})

In [ ]:
# Load Physician claim relationships
for role in physician_role_map.values():
    print(f'Loading {role} relationships...')
    gds.run_cypher(
        f"""
          UNWIND $records AS row
          MATCH (physician:Physician {{uid: row.physician_id}})
          MATCH (claim:Claim {{uid: row.claim_id}})
          MERGE (physician)<-[:{role}]-(claim)
        """
        , params={'records': physician_df.loc[physician_df.physician_role == role, ['claim_id','physician_id']].to_dict('records')})
print('Loading complete')

Then create `AdmitDiagnosis` and `DiagnosisGroup` nodes and relate them to Claims and Providers as well

In [ ]:
#load admin diagnosis codes
print(f'Loading AdmitDiagnosis nodes...')
gds.run_cypher(
    """
      UNWIND $claim_admit_diagnosis_codes as claim_admit_diagnosis_code
      MERGE (admitDiagnosis:AdmitDiagnosis {uid: claim_admit_diagnosis_code})
    """
    , params={'claim_admit_diagnosis_codes': raw_df.claim_admit_diagnosis_code.dropna().drop_duplicates().to_list()})

#load admin diagnosis relationships
print(f'Loading HAS_ADMIT_DIAGNOSIS relationships...')
gds.run_cypher(
    """
      UNWIND $records AS row
      MATCH (claim:Claim {uid: row.claim_id})
      MATCH (admitDiagnosis:AdmitDiagnosis {uid: row.claim_admit_diagnosis_code})
      MERGE (claim)-[:HAS_ADMIT_DIAGNOSIS]->(admitDiagnosis)
    """
    , params={'records': raw_df[['claim_id','claim_admit_diagnosis_code']].dropna().to_dict('records')})
print('Loading complete')

In [ ]:
#load admin diagnosis codes
print(f'Loading DiagnosisGroup nodes...')
gds.run_cypher(
    """
      UNWIND $diag_group_codes as diag_group_code
      MERGE (diagnosisGroup:DiagnosisGroup {uid: diag_group_code})
    """
    , params={'diag_group_codes': raw_df.diag_group_code.dropna().drop_duplicates().to_list()})

#load admin diagnosis relationships
print(f'Loading HAS_DIAGNOSIS_GROUP relationships...')
gds.run_cypher(
    """
      UNWIND $records AS row
      MATCH (claim:Claim {uid: row.claim_id})
      MATCH (diagnosisGroup:DiagnosisGroup {uid: row.diag_group_code})
      MERGE (claim)-[:HAS_DIAGNOSIS_GROUP]->(diagnosisGroup)
    """
    , params={'records': raw_df[['claim_id','diag_group_code']].dropna().to_dict('records')})
print('Loading complete')

As per the schema, Claims are related to procedures. Let deal with those nodes & relationships now

In [ ]:
# create a reshaped procedure dataframe to make loading more efficient
proc_cols = [col for col in raw_df.columns if 'claim_procedure_code' in col]
proc_df = pd.wide_to_long(raw_df[proc_cols  + ['claim_id']], stubnames='claim_procedure_code', i='claim_id', j='proc', sep='_').dropna().reset_index()
proc_df.claim_procedure_code = pd.to_numeric(proc_df.claim_procedure_code).astype('Int64')
proc_df

In [ ]:
# load procedure nodes
print(f'Loading Procedure nodes...')
gds.run_cypher(
    """
      UNWIND $claim_procedure_codes as claim_procedure_code
      MERGE (proc:Procedure {uid: claim_procedure_code})
    """
    , params={'claim_procedure_codes': proc_df.claim_procedure_code.drop_duplicates().to_list()})

#load procedure - claim relationships
print(f'Loading HAS_PROCEDURE relationships...')
gds.run_cypher(
    """
      UNWIND $records AS row
      MATCH (claim:Claim {uid: row.claim_id})
      MATCH (proc:Procedure {uid: row.claim_procedure_code})
      MERGE (claim)-[:HAS_PROCEDURE]->(proc)
    """
    , params={'records': proc_df.to_dict('records')})
print('Loading complete')

As per the schema, Claims also have diagnoses. Let deal with those nodes & relationships here too

In [ ]:
# create a reshaped claim diagnosis dataframe to make loading more efficient
diag_cols = [col for col in raw_df.columns if 'claim_diag_code' in col]
claim_diag_df = pd.wide_to_long(raw_df[diag_cols  + ['claim_id']], stubnames='claim_diag_code', i='claim_id', j='claim_number', sep='_').dropna().reset_index()
claim_diag_df

In [ ]:
# load diagnosis claim nodes
print(f'Loading Diagnosis nodes...')
gds.run_cypher(
    """
      UNWIND $claim_diag_code as claim_diag_code
      MERGE (diag:Diagnosis {uid: claim_diag_code})
    """
    , params={'claim_diag_code': claim_diag_df.claim_diag_code.drop_duplicates().to_list()})

#load diagnosis - claim relationships
print(f'Loading HAS_DIAGNOSIS relationships...')
gds.run_cypher(
    """
      UNWIND $records AS row
      MATCH (claim:Claim {uid: row.claim_id})
      MATCH (diag:Diagnosis {uid: row.claim_diag_code})
      MERGE (claim)-[:HAS_DIAGNOSIS]->(diag)
    """
    , params={'records': claim_diag_df.to_dict('records')})
print('Loading complete')

Finally, lets connect our Benificiaries (or) patients with diseases they suffer from

In [ ]:
# create a reshaped chronic condition dataframe to make loading more efficient
cond_cols = [col for col in raw_df.columns if 'chronic_cond' in col]
cond_df = (pd.wide_to_long(raw_df[cond_cols  + ['bene_id']].drop_duplicates(), stubnames='chronic_cond', i='bene_id', j='condition', sep='_', suffix='\D+')
           .reset_index().query('chronic_cond == "1"').drop(columns='chronic_cond'))
cond_df

In [ ]:
# load condition nodes
print(f'Loading Condition nodes...')
gds.run_cypher(
    """
      UNWIND $conditions as condition
      MERGE (cond:Condition {name:condition})
    """
    , params={'conditions': cond_df.condition.drop_duplicates().to_list()})

#load condition - beneficiary relationships
print(f'Loading HAS_CHRONIC relationships...')
gds.run_cypher(
    """
      UNWIND $records AS row
      MATCH (bene:Beneficiary {uid: row.bene_id})
      MATCH (cond:Condition {name:row.condition})
      MERGE (bene)-[:HAS_CHRONIC]->(cond)
    """
    , params={'records': cond_df.to_dict('records')})
print('Loading complete')

In [ ]:
#Load renal disease indicator. Unlike above this isn't necessarily associated with a chronic condition
gds.run_cypher('MERGE (cond:Condition {name:"renaldisease (from indicator)"})')

#load renal disease - beneficiary relationships
print(f'Loading HAS_DISEASE relationships...')
gds.run_cypher(
    """
      UNWIND $bene_ids AS bene_id
      MATCH (bene:Beneficiary {uid: bene_id})
      MATCH (cond:Condition {name:"renaldisease (from indicator)"})
      MERGE (bene)-[:HAS_DISEASE]->(cond)
    """
    , params={'bene_ids': raw_df.loc[raw_df.renal_disease_indicator == '1', 'bene_id'].drop_duplicates().tolist()})
print('Loading complete')

Below is a breakdown of high-level counts by labels and relationships

In [ ]:
# total node counts
gds.run_cypher("""
    CALL apoc.meta.stats()
    YIELD labels
    UNWIND keys(labels) AS nodeLabel
    RETURN nodeLabel, labels[nodeLabel] AS nodeCount
""")

In [ ]:
# total relationship counts
gds.run_cypher("""
    CALL apoc.meta.stats()
    YIELD relTypesCount
    UNWIND keys(relTypesCount) AS relationshipType
    RETURN relationshipType, relTypesCount[relationshipType] AS relationshipCount
""")

## Graph Data Science
We got the data inside our Database! Let's do some Graph Data Science. This is how a typical GDS workflow looks like inside Neo4j

![GDS Workflow](https://storage.googleapis.com/neo4j-datasets/insurance-claim/img/gds_workflow.png)

As first step, we're going to use Neo4j Graph Data Science to create an in memory graph represtation of the data.  We'll enhance that representation with features we engineer using a graph embedding.

In [242]:
g, _ = gds.graph.project('projection',
                         ['Beneficiary','Condition','County','State','Claim','DiagnosisGroup','AdmitDiagnosis',
                          'Physician','Provider','Diagnosis','Procedure'],
                         {
                             'LOCATED_AT': {'orientation': 'UNDIRECTED'},
                             'FILED_CLAIM': {'orientation': 'UNDIRECTED'},
                             'PROVIDED_BY': {'orientation': 'UNDIRECTED'},
                             'PART_OF': {'orientation': 'UNDIRECTED'},
                             'WORKS_FOR': {'orientation': 'UNDIRECTED'},
                             'ATTENDED_BY': {'orientation': 'UNDIRECTED'},
                             'OPERATED_BY': {'orientation': 'UNDIRECTED'},
                             'ALSO_ATTENDED_BY': {'orientation': 'UNDIRECTED'},
                             'HAS_ADMIT_DIAGNOSIS': {'orientation': 'UNDIRECTED'},
                             'HAS_DIAGNOSIS_GROUP': {'orientation': 'UNDIRECTED'},
                             'HAS_PROCEDURE': {'orientation': 'UNDIRECTED'},
                             'HAS_DIAGNOSIS': {'orientation': 'UNDIRECTED'},
                             'HAS_CHRONIC': {'orientation': 'UNDIRECTED'},
                             'HAS_DISEASE': {'orientation': 'UNDIRECTED'}
                         })
_

nodeProjection            {'Condition': {'label': 'Condition', 'properti...
relationshipProjection    {'HAS_DIAGNOSIS': {'orientation': 'UNDIRECTED'...
graphName                                                        projection
nodeCount                                                            167271
relationshipCount                                                   2105376
projectMillis                                                           297
Name: 0, dtype: object

If you get an error saying the graph already exists, that's probably because you ran this code before. You can destroy it using this command:

In [ ]:
# if gds.graph.exists('projection').exists:
#    gds.graph.get('projection').drop()

Now, let's list the details of the graph to make sure the projection was created as we want.

In [247]:
print(f"node count: {g.node_count()}")
print(f"relationship count: {g.relationship_count()}")
print(f"node labels: {g.node_labels()}")
print(f"relationship types: {g.relationship_types()}")

node count: 167271
relationship count: 2105376
node labels: ['Condition', 'State', 'DiagnosisGroup', 'Physician', 'Procedure', 'Diagnosis', 'AdmitDiagnosis', 'County', 'Claim', 'Beneficiary', 'Provider']
relationship types: ['HAS_DIAGNOSIS', 'HAS_PROCEDURE', 'HAS_DIAGNOSIS_GROUP', 'OPERATED_BY', 'PART_OF', 'PROVIDED_BY', 'ATTENDED_BY', 'LOCATED_AT', 'HAS_DISEASE', 'HAS_ADMIT_DIAGNOSIS', 'FILED_CLAIM', 'ALSO_ATTENDED_BY', 'WORKS_FOR', 'HAS_CHRONIC']


Now we can generate an embedding from that graph. This is a new feature we can use in our predictions. We're using FastRP, which is a more full featured and higher performance of Node2Vec. You can learn more about that [here](https://neo4j.com/docs/graph-data-science/current/algorithms/fastrp/).

There are a bunch of parameters we could adjust in this.  One of the most obvious is the embeddingDimension.  The documentation covers many more.

In [248]:
gds.fastRP.mutate(g, embeddingDimension=32, randomSeed=1, mutateProperty='embedding')

nodePropertiesWritten                                               167271
mutateMillis                                                             0
nodeCount                                                           167271
preProcessingMillis                                                      0
computeMillis                                                          369
configuration            {'nodeSelfInfluence': 0, 'propertyRatio': 0.0,...
Name: 0, dtype: object

That creates an embedding for each node type.  However, we only want the embedding on the nodes of type holding.

We're going to take the embedding from our projection and write it to the holding nodes in the underlying database.

In [253]:
result = gds.graph.writeNodeProperties(g, ['embedding'], ['Claim'])
result.writeMillis

176

In [258]:
result_df = gds.run_cypher(
  """ 
    MATCH (claim:Claim)
    RETURN claim.uid as id, claim.embedding as embedding, claim.isFraud as target
  """
)
result_df

,id,embedding,target
0,CLM110011,"[-0.3046455681324005, -0.48370498418807983, -0...",1
1,CLM110030,"[0.09364302456378937, 0.013624608516693115, -0...",1
2,CLM110031,"[-0.2793812155723572, -0.6372988820075989, 0.0...",1
3,CLM110038,"[-0.07748548686504364, -0.25101712346076965, 0...",0
4,CLM110040,"[-0.362213134765625, -0.22709669172763824, -0....",1
...,...,...,...
100075,CLM82009,"[0.04165990650653839, -0.6641069650650024, -0....",1
100076,CLM82013,"[-0.0238773375749588, -0.6360070109367371, -0....",1
100077,CLM82218,"[0.12991547584533691, -0.24101734161376953, 0....",1
100078,CLM82303,"[-0.054207198321819305, -0.005985379219055176,...",1


This is what we just did

![embeddings](https://storage.googleapis.com/neo4j-datasets/insurance-claim/img/what_are_embeddings.png)

# Export Embeddings
Now we're going to reformat the query output so that the embeddings can be fed in to a Vertex AI Auto ML pipeline. Note that we are exporting only embeddings and all the other features are intentionally left out. This is to showcase how powerful these vectors are!

Note that the embedding row is an array. To make this dataset more consumable, we should flatten that out into multiple individual features: embedding_0, embedding_1, ... embedding_n.


In [260]:
embedding_df = pd.DataFrame(result_df['embedding'].values.tolist()).add_prefix("embedding_")
merged_df = result_df.drop(columns=['embedding']).merge(embedding_df, left_index=True, right_index=True)
merged_df

,id,target,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,...,embedding_22,embedding_23,embedding_24,embedding_25,embedding_26,embedding_27,embedding_28,embedding_29,embedding_30,embedding_31
0,CLM110011,1,-0.304646,-0.483705,-0.357203,-0.302861,0.100711,0.106431,0.021861,-0.274979,...,-0.328284,0.077383,0.046006,-0.056047,-0.129007,-0.007562,0.271084,0.115958,-0.175314,0.513049
1,CLM110030,1,0.093643,0.013625,-0.479409,0.057462,-0.219684,-0.084565,-0.375288,0.117837,...,-0.214706,0.268210,0.220086,0.080279,-0.077321,0.318584,0.101539,0.018138,0.171507,0.077746
2,CLM110031,1,-0.279381,-0.637299,0.007481,0.098682,0.481290,-0.065113,0.280593,-0.153458,...,-0.092117,0.524109,0.339914,-0.375202,-0.041442,0.231110,0.331119,0.129880,0.049741,-0.284475
3,CLM110038,0,-0.077485,-0.251017,0.180458,0.633138,0.317620,0.253783,-0.156635,0.126405,...,0.028823,0.170212,0.731474,-0.216520,0.031700,0.159313,0.425701,0.008861,0.292919,-0.360491
4,CLM110040,1,-0.362213,-0.227097,-0.192352,0.089990,-0.015110,-0.441792,-0.021205,0.137845,...,-0.407148,-0.137814,-0.271657,-0.010866,-0.178115,0.233506,0.160516,0.396605,-0.001921,0.078454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100075,CLM82009,1,0.041660,-0.664107,-0.249968,0.591536,-0.208832,0.183202,-0.469215,0.017440,...,0.051976,0.416909,0.158655,0.014122,-0.190709,0.159962,0.401225,-0.089189,0.109903,-0.005102
100076,CLM82013,1,-0.023877,-0.636007,-0.467070,0.062771,0.472643,-0.110822,-0.589941,-0.016301,...,0.388619,-0.045963,0.019847,-0.097595,-0.222180,-0.005405,-0.111435,0.218153,0.035548,0.387169
100077,CLM82218,1,0.129915,-0.241017,0.205425,0.469056,0.049277,-0.411168,-0.260873,-0.494940,...,0.109164,0.040084,0.152925,0.002155,-0.213440,0.136800,0.433116,0.312274,-0.097761,0.267573
100078,CLM82303,1,-0.054207,-0.005985,-0.224177,0.502715,0.030636,0.002026,-0.081165,-0.202303,...,0.131457,-0.247158,0.555261,0.253013,0.278891,0.391433,0.137198,-0.133029,0.468623,0.033940


If you are curious, visualize the embeddings as a t-SNE plot. It can look something like this:
![embedding_viz](https://storage.googleapis.com/neo4j-datasets/insurance-claim/img/embeddings-tsne.png)

Now that we have the data formatted properly, let's write it as CSV

In [261]:
import os

outdir = './data'
if not os.path.exists(outdir):
    os.mkdir(outdir)

data = merged_df.sample(frac=1).reset_index(drop=True)
data.to_csv(os.path.join(outdir, 'embedding.csv'), index=False)

## Upload to Google Cloud Storage
Now let's write the file to Google Cloud Storage so we can use it in our model.  To do so, we must set a few environment variables.

Edit the REGION variable below.  You'll want to be sure it matches the region where your notebook is running.

The STORAGE_BUCKET is the name of a new bucket.  It must be globally unique.  It also needs to be all lower case.

In [ ]:
import os

# Edit this variable!
REGION = 'us-west1'
shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
PROJECT_ID = shell_output[0]

STORAGE_BUCKET = PROJECT_ID + '-fsi'
STORAGE_BUCKET

os.environ["GCLOUD_PROJECT"] = PROJECT_ID

In [ ]:
from google.cloud import storage
client = storage.Client()
bucket = client.bucket(STORAGE_BUCKET)
if not bucket.exists:
    bucket.create(location=REGION)

blob = bucket.blob(os.path.join('insurance_fraud', 'embedding.csv'))
blob.upload_from_filename(os.path.join(outdir, 'embedding.csv'))

We will also export factorize the raw data without embeddings and run a similar Auto ML pipeline on Vertex AI. Then, lets compare the accuracy between the two!

In [266]:
benchmark_df = raw_df.rename(columns={'claim_id': 'id', 'is_fraud': 'target'})
for col in benchmark_df.columns:
    if ('claim_diag_code' in col) or ('claim_procedure_code' in col) or (col in ['claim_admit_diagnosis_code', 'diag_group_code']):
        benchmark_df[col] = pd.factorize(benchmark_df[col])[0] + 1
benchmark_df

,bene_id,id,claim_start_dt,claim_end_dt,provider,claim_amt_reimbursed,attending_physician,operating_physician,other_physician,claim_diag_code_1,...,chronic_cond_diabetes,chronic_cond_ischemicheart,chronic_cond_osteoporasis,chronic_cond_rheumatoidarthritis,chronic_cond_stroke,ip_annual_reimbursement_amt,ip_annual_deductible_amt,op_annual_reimbursement_amt,op_annual_deductible_amt,target
0,BENE11002,CLM624349,2009-10-11,2009-10-11,PRV56011,30,PHY326117,NaN,NaN,1,...,1,1,1,1,1,0,0,30,50,1
1,BENE11004,CLM121801,2009-01-06,2009-01-06,PRV56011,40,PHY334319,NaN,NaN,2,...,0,0,0,0,1,0,0,1810,760,1
2,BENE11004,CLM150998,2009-01-22,2009-01-22,PRV56011,200,PHY403831,NaN,NaN,3,...,0,0,0,0,1,0,0,1810,760,1
3,BENE11004,CLM173224,2009-02-03,2009-02-03,PRV56011,20,PHY339887,NaN,NaN,4,...,0,0,0,0,1,0,0,1810,760,1
4,BENE11004,CLM224741,2009-03-03,2009-03-03,PRV56011,40,PHY345721,NaN,NaN,5,...,0,0,0,0,1,0,0,1810,760,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100075,BENE100433,CLM514796,2009-08-08,2009-08-09,PRV54895,10,PHY382394,NaN,NaN,1815,...,0,0,1,1,1,0,0,1630,350,1
100076,BENE100499,CLM237382,2009-03-09,2009-03-10,PRV54895,400,PHY385898,NaN,NaN,1351,...,1,0,0,1,0,0,0,3870,970,1
100077,BENE100499,CLM676967,2009-11-11,2009-11-11,PRV54895,50,PHY385898,NaN,NaN,915,...,1,0,0,1,0,0,0,3870,970,1
100078,BENE100569,CLM602032,2009-09-27,2009-09-27,PRV54895,500,PHY368638,PHY368638,NaN,128,...,1,0,1,1,1,0,0,1000,700,1


In [ ]:
benchmark_sample_df = benchmark_df.sample(frac=1).reset_index(drop=True)
benchmark_sample_df.to_csv(os.path.join(outdir, 'raw.csv'), index=False)

blob = bucket.blob(os.path.join('insurance_fraud', 'raw.csv'))
blob.upload_from_filename(os.path.join(outdir, 'raw.csv'))